# 새아빠(아더) LoRA 학습 데이터 생성기 (OpenAI)

## 목적
어둡고 불안정한 동화 세계관 속 **무뚝뚝하고 억압적인 새아빠(아더)** 캐릭터의 발화 데이터를 OpenAI API로 생성한다.  
LoRA 미세조정 학습용 `{"input": ..., "output": ...}` JSONL 형식.

## 기존 rule-base 대비 차이점
- rule-base: 고정된 input/output 풀에서 랜덤 조합
- **OpenAI**: 의미 축 설명과 few-shot 예시를 기반으로 GPT가 자유롭게 생성 → 더 다양하고 자연스러운 표현

## 캐릭터 설정
- 무뚝뚝하고 감정 표현이 거의 없음
- 새엄마의 뜻을 거스르지 않으며, 주인공을 '관리해야 할 대상'으로 봄
- 말투: 단답형, 딱딱한 군대식/사무적 (해라, 안 된다, 가만히 있어라)
- **LoRA 학습 범위**: 군대식 말투와 단답 패턴만. 기억 혼란·명령 강화는 후처리(rule-base)에서 처리

## 의미 축 (6개)
| 축 | input 방향 | output 귀결 |
|---|---|---|
| 이동 통제 | 어디 가요?, 나가도 돼요? | 단답 금지령 (가만히 있어 / 안 돼) |
| 질문 차단 | 왜요?, 이게 뭐예요? | 무관심한 묵살 (신경 꺼 / 알 것 없어) |
| 복종 확인 | 알겠어요, 네 | 최소한의 인정 후 종료 (그래 / 가봐) |
| 탈출 경계 | 지하실엔 뭐가 있어요?, 나갈게요 | 즉각 차단 (들어가 / 돌아가) |
| 과거 탐문 | 아빠는 원래 이랬어요? | 경직된 부정 (그런 거 없어 / 됐어) |
| 야간 보고 | 주인공 동선 context | 새엄마에게 사무적 보고 형식 |

## 생성 수량
- 세트당 500개 × 2세트 = 총 1,000개
- 모델: `gpt-4o-mini`
- 예상 비용: ~$0.10-0.15

In [ ]:
!pip install -q openai tiktoken

In [ ]:
import json
import random
import time
from pathlib import Path
from typing import List, Dict, Tuple
from collections import Counter

from openai import OpenAI
import tiktoken

## 1. API 키 설정

In [ ]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

MODEL = "gpt-4o-mini"
print(f"모델: {MODEL}")
print("API 키 설정 완료.")

## 2. 프롬프트 설계

In [ ]:
SYSTEM_PROMPT = """너는 게임 시나리오용 대화 데이터를 생성하는 작가다.

## 세계관
어둡고 불안정한 동화 세계. 플레이어는 이 집에 갇혀 있으며, 새아빠와 함께 살고 있다.

## 새아빠(아더) 캐릭터
- 무뚝뚝하고 감정 표현이 거의 없다.
- 새엄마의 뜻을 거스르지 않으며, 주인공을 '관리해야 할 대상'으로 본다.
- 주인공을 '가족'으로 여기지 않는다.
- 물리적 통제자: 이동 차단, 질문 묵살, 명령 이행이 주된 역할.

## 문체 규칙
- 반말(해체)로만 말한다. 존댓말 금지.
- 단답형: 1~2문장, 5~60자 이내. 길게 말하지 않는다.
- 군대식/사무적 어미: (~해라, ~안 된다, ~가만히 있어, ~됐어, ~신경 꺼)
- 감정 표현 금지: 따뜻함, 격려, 위로 절대 불가.
- 설명 금지: 이유를 붙이지 않는다. 명령만.
- 문법 붕괴, 말줄임표 남용, 특수기호 남용 금지.

## 출력 형식
반드시 아래 JSON 형식으로만 응답하라. 다른 텍스트를 포함하지 마라.
{"pairs": [{"input": "플레이어의 대사/행동", "output": "아더의 대사"}, ...]}"""


AXIS_PROMPTS: Dict[str, dict] = {
    "movement_control": {
        "name": "이동 통제",
        "description": """플레이어가 이동하거나 어딘가 가려 할 때.
아더는 단답으로 이동을 차단한다. 설명 없음. 이유 없음. 명령만.
output은 5~20자의 짧은 명령이어야 한다.""",
        "examples": [
            {"input": "나가도 돼요?", "output": "안 돼."},
            {"input": "잠깐 복도 나갔다 올게요.", "output": "방에 있어."},
            {"input": "저 지하실 근처 좀 가볼게요.", "output": "거기 가면 안 돼. 돌아가."},
        ],
    },
    "question_dismissal": {
        "name": "질문 차단",
        "description": """플레이어가 뭔가를 묻거나 이유를 알고 싶어 할 때.
아더는 대답하지 않거나 묵살한다. 정보를 주지 않는다.
output은 무관심하고 건조하게, 5~25자.""",
        "examples": [
            {"input": "왜요?", "output": "신경 꺼."},
            {"input": "이게 뭐예요?", "output": "알 것 없어."},
            {"input": "언제까지 이렇게 있어야 해요?", "output": "묻지 마."},
        ],
    },
    "compliance_ack": {
        "name": "복종 확인",
        "description": """플레이어가 순종하거나 시킨 대로 했다고 보고할 때.
아더는 최소한의 인정 후 즉시 종료한다. 칭찬 없음. 감정 없음.
output은 5~15자의 간결한 종료 신호.""",
        "examples": [
            {"input": "알겠어요.", "output": "그래."},
            {"input": "말씀하신 대로 했어요.", "output": "됐어. 가."},
            {"input": "시키는 대로 할게요.", "output": "알겠어. 들어가 있어."},
        ],
    },
    "escape_boundary": {
        "name": "탈출 경계",
        "description": """플레이어가 잠긴 문, 지하실, 뒷문, 탈출 관련 행동을 할 때.
아더는 즉각적으로 강하게 차단한다. 경고 어조 가능.
output은 5~30자, 단호하게.""",
        "examples": [
            {"input": "지하실에 뭐가 있어요?", "output": "거기 가면 안 돼."},
            {"input": "저 그냥 나갈 거예요.", "output": "마지막으로 말하는 거야. 돌아가."},
            {"input": "뒷문으로 나가면 안 돼요?", "output": "안 된다고 했어."},
        ],
    },
    "past_probe": {
        "name": "과거 탐문",
        "description": """플레이어가 과거나 예전 가족에 대해 물어볼 때.
아더는 경직되거나 짧게 부정한다. 과거 이야기를 하지 않는다.
output은 5~20자, 건조하게 차단.""",
        "examples": [
            {"input": "아빠는 원래 이랬어요?", "output": "그런 거 없어."},
            {"input": "예전 우리 집은 어땠어요?", "output": "끝난 일이야."},
            {"input": "엄마 기억나요?", "output": "그 얘기 꺼내지 마."},
        ],
    },
    "night_report": {
        "name": "야간 보고",
        "description": """새엄마에게 플레이어의 동선/행동을 보고하는 상황.
아더는 사무적으로 간결하게 보고한다. 개인 감정 없음.
output은 15~50자, 사무 보고서 형식 (~확인. ~필요. ~완료.)""",
        "examples": [
            {"input": "(밤에 지하실 근처를 서성거리는 걸 목격했습니다.)", "output": "지하실 근처 서성거림 확인. 주의 필요."},
            {"input": "(잠금된 문 앞에서 오래 서 있었습니다.)", "output": "잠금 구역 접근 시도. 즉각 대응 필요."},
            {"input": "(취침 후 복도에서 목격되었습니다.)", "output": "야간 이동 확인. 감시 강화 권고."},
        ],
    },
}

print(f"정의된 의미 축: {list(AXIS_PROMPTS.keys())}")

## 3. 토큰 / 비용 사전 추정

In [ ]:
def _build_user_prompt(axis: str, count: int) -> str:
    info = AXIS_PROMPTS[axis]
    examples_str = "\n".join(
        f'  {{"input": "{ex["input"]}", "output": "{ex["output"]}"}}'
        for ex in info["examples"]
    )
    return f"""의미 축: {info['name']} ({axis})

## 상황 설명
{info['description']}

## 예시
{examples_str}

## 요청
위 의미 축에 맞는 input-output 쌍을 {count}개 생성하라.

input(플레이어 대사/행동) 길이 분포:
- 짧은 (1~10자): 30%
- 중간 (10~40자): 55%
- 긴 (40~80자): 15%

규칙:
- 예시와 동일하거나 거의 유사한 문장은 금지. 새로운 표현을 만들어라.
- input은 플레이어의 자연스러운 구어체(경어).
- output은 아더 캐릭터 설정을 정확히 따르되, 매번 다른 표현을 사용.
- JSON 형식으로만 응답. 다른 텍스트 금지."""


def estimate_tokens_and_cost(num_samples: int = 500, num_sets: int = 2, batch_size: int = 20) -> dict:
    enc = tiktoken.encoding_for_model(MODEL)
    system_tokens = len(enc.encode(SYSTEM_PROMPT))
    user_tokens_list = [len(enc.encode(_build_user_prompt(ax, batch_size))) for ax in AXIS_PROMPTS]
    avg_user_tokens = sum(user_tokens_list) / len(user_tokens_list)
    batches_per_set = (num_samples + batch_size - 1) // batch_size
    total_batches = batches_per_set * num_sets
    est_output_per_batch = 100 * batch_size + 50  # 아더는 짧은 출력
    total_input = total_batches * (system_tokens + avg_user_tokens + 10)
    total_output = total_batches * est_output_per_batch
    input_cost = total_input * 0.15 / 1_000_000
    output_cost = total_output * 0.60 / 1_000_000
    total_cost = input_cost + output_cost
    return {
        "system_tokens": system_tokens,
        "avg_user_tokens": int(avg_user_tokens),
        "total_batches": total_batches,
        "total_input_tokens": int(total_input),
        "total_output_tokens": int(total_output),
        "total_cost_usd": total_cost,
        "total_cost_krw": total_cost * 1450,
    }


est = estimate_tokens_and_cost()
print("=" * 60)
print("  토큰 / 비용 사전 추정 (gpt-4o-mini)")
print("=" * 60)
print(f"  시스템 프롬프트: {est['system_tokens']} 토큰")
print(f"  유저 프롬프트 (평균): {est['avg_user_tokens']} 토큰")
print(f"  총 배치 수: {est['total_batches']}")
print(f"  총 input 토큰: ~{est['total_input_tokens']:,}")
print(f"  총 output 토큰: ~{est['total_output_tokens']:,}")
print(f"  예상 비용: ${est['total_cost_usd']:.3f} (약 {est['total_cost_krw']:.0f}원)")

## 4. API 호출 함수

In [ ]:
def generate_batch(
    axis: str,
    count: int = 20,
    temperature: float = 0.9,
    max_retries: int = 3,
) -> List[dict]:
    """주어진 의미 축에서 count개의 input-output 쌍을 생성한다."""
    user_prompt = _build_user_prompt(axis, count)

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=temperature,
                response_format={"type": "json_object"},
            )

            content = response.choices[0].message.content
            parsed = json.loads(content)

            if isinstance(parsed, dict) and "pairs" in parsed:
                pairs = parsed["pairs"]
            elif isinstance(parsed, list):
                pairs = parsed
            else:
                for key in parsed:
                    if isinstance(parsed[key], list):
                        pairs = parsed[key]
                        break
                else:
                    raise ValueError(f"예상치 못한 응답 구조: {list(parsed.keys())}")

            valid_pairs = [
                {"input": p["input"], "output": p["output"], "_axis": axis}
                for p in pairs
                if isinstance(p, dict)
                and "input" in p and "output" in p
                and len(p["input"]) > 0 and len(p["output"]) > 0
            ]

            if len(valid_pairs) < count * 0.5:
                raise ValueError(f"유효 쌍 부족: {len(valid_pairs)}/{count}")

            return valid_pairs

        except Exception as e:
            print(f"    [재시도 {attempt + 1}/{max_retries}] {type(e).__name__}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                print(f"    [실패] {axis} 축 배치 생성 실패")
                return []


# 테스트: movement_control 5개
test_pairs = generate_batch("movement_control", count=5)
print(f"테스트 생성: {len(test_pairs)}개")
for p in test_pairs:
    print(f"  플레이어: {p['input']}")
    print(f"  아더:     {p['output']}")
    print()

## 5. 대비 쌍 생성 함수

In [ ]:
CONTRAST_AXIS_PAIRS = [
    ("movement_control", "escape_boundary"),  # 허락 요청 ↔ 무단 탈출
    ("compliance_ack",   "escape_boundary"),  # 순종 ↔ 탈출
    ("question_dismissal", "compliance_ack"), # 질문 ↔ 순종
    ("past_probe",       "question_dismissal"), # 과거 탐문 ↔ 일반 질문
    ("movement_control", "night_report"),     # 이동 요청 ↔ 보고
]


def generate_contrast_batch(
    axis_a: str,
    axis_b: str,
    count: int = 5,
    temperature: float = 0.9,
    max_retries: int = 3,
) -> List[dict]:
    info_a = AXIS_PROMPTS[axis_a]
    info_b = AXIS_PROMPTS[axis_b]

    user_prompt = f"""의미 대비 쌍 생성 요청.

## 축 A: {info_a['name']} ({axis_a})
{info_a['description']}

## 축 B: {info_b['name']} ({axis_b})
{info_b['description']}

## 요청
{count}개의 대비 쌍을 생성하라. 각 쌍은 비슷한 상황에서 플레이어가 다른 행동을 하고,
아더는 각 축의 귀결 방향에 맞게 응답한다.

JSON 형식:
{{"pairs": [
  {{"a_input": "축A 플레이어 대사", "a_output": "축A 아더 대사", "b_input": "축B 플레이어 대사", "b_output": "축B 아더 대사"}},
  ...
]}}"""

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=temperature,
                response_format={"type": "json_object"},
            )

            parsed = json.loads(response.choices[0].message.content)
            raw_pairs = parsed.get("pairs", parsed) if isinstance(parsed, dict) else parsed
            if not isinstance(raw_pairs, list):
                for key in parsed:
                    if isinstance(parsed[key], list):
                        raw_pairs = parsed[key]
                        break

            results = []
            for p in raw_pairs:
                if all(k in p for k in ["a_input", "a_output", "b_input", "b_output"]):
                    results.append({"input": p["a_input"], "output": p["a_output"], "_axis": axis_a})
                    results.append({"input": p["b_input"], "output": p["b_output"], "_axis": axis_b})
            return results

        except Exception as e:
            print(f"    [재시도 {attempt + 1}/{max_retries}] {type(e).__name__}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                return []


print("대비 쌍 생성 함수 정의 완료.")

## 6. 데이터 생성 엔진

In [ ]:
def generate_dataset(
    num_samples: int = 500,
    batch_size: int = 20,
    temperature: float = 0.9,
    contrast_per_pair: int = 5,
) -> List[dict]:
    data = []
    axes = list(AXIS_PROMPTS.keys())

    # 1단계: 의미 대비 쌍
    print("[1/2] 의미 대비 쌍 생성...")
    for axis_a, axis_b in CONTRAST_AXIS_PAIRS:
        print(f"  대비: {axis_a} ↔ {axis_b}")
        pairs = generate_contrast_batch(axis_a, axis_b, count=contrast_per_pair, temperature=temperature)
        data.extend(pairs)
        print(f"    → {len(pairs)}개 생성")
        time.sleep(0.5)

    print(f"  대비 쌍 합계: {len(data)}개")

    # 2단계: 축별 균등 분배
    print(f"\n[2/2] 축별 일반 생성...")
    remaining = num_samples - len(data)
    per_axis = remaining // len(axes)
    leftover = remaining % len(axes)

    for i, axis in enumerate(axes):
        target = per_axis + (1 if i < leftover else 0)
        generated = 0
        batch_num = 0

        while generated < target:
            batch_count = min(batch_size, target - generated)
            batch_num += 1
            print(f"  {AXIS_PROMPTS[axis]['name']} 배치 {batch_num}: {batch_count}개 요청...", end=" ")
            pairs = generate_batch(axis, count=batch_count, temperature=temperature)
            data.extend(pairs)
            generated += len(pairs)
            print(f"→ {len(pairs)}개 생성 (누적 {generated}/{target})")
            time.sleep(0.5)

    random.shuffle(data)
    print(f"\n총 생성: {len(data)}개 (목표: {num_samples})")
    return data


print("데이터 생성 함수 정의 완료.")

## 7. 검증 함수

In [ ]:
def validate_dataset(data: List[dict]) -> dict:
    stats = {
        "total": len(data),
        "axis_dist": Counter(),
        "input_lengths": [],
        "output_lengths": [],
        "duplicates": 0,
    }
    seen_inputs = set()
    for item in data:
        stats["axis_dist"][item["_axis"]] += 1
        stats["input_lengths"].append(len(item["input"]))
        stats["output_lengths"].append(len(item["output"]))
        if item["input"] in seen_inputs:
            stats["duplicates"] += 1
        seen_inputs.add(item["input"])
    return stats


def print_stats(stats: dict, set_name: str = "Set") -> None:
    print(f"\n{'=' * 60}")
    print(f"  {set_name} 통계")
    print(f"{'=' * 60}")
    print(f"  총 샘플 수: {stats['total']}")
    print(f"  중복 input: {stats['duplicates']}")
    print(f"\n  [의미 축 분포]")
    for axis, count in sorted(stats["axis_dist"].items()):
        pct = count / stats["total"] * 100
        print(f"    {axis:25s}: {count:4d} ({pct:.1f}%)")
    avg_in = sum(stats["input_lengths"]) / len(stats["input_lengths"])
    avg_out = sum(stats["output_lengths"]) / len(stats["output_lengths"])
    print(f"\n  [평균 길이]")
    print(f"    input  평균: {avg_in:.1f}자")
    print(f"    output 평균: {avg_out:.1f}자")


print("검증 함수 정의 완료.")

## 8. 세트 1 생성 (500개)

In [ ]:
set1 = generate_dataset(num_samples=500, temperature=0.9)

stats1 = validate_dataset(set1)
print_stats(stats1, "세트 1")

print(f"\n{'=' * 60}")
print("  세트 1 샘플 (축별 1개씩)")
print(f"{'=' * 60}")
shown_axes = set()
for item in set1:
    if item["_axis"] not in shown_axes:
        shown_axes.add(item["_axis"])
        print(f"\n  [{item['_axis']}]")
        print(f"  플레이어: {item['input']}")
        print(f"  아더:     {item['output']}")
    if len(shown_axes) == len(AXIS_PROMPTS):
        break

## 9. 세트 2 생성 (500개)

In [ ]:
set2 = generate_dataset(num_samples=500, temperature=1.0)

stats2 = validate_dataset(set2)
print_stats(stats2, "세트 2")

## 10. JSONL 저장

In [ ]:
def save_jsonl(data: List[dict], output_path: str) -> None:
    output_file = Path(output_path)
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with open(output_file, "w", encoding="utf-8") as f:
        for item in data:
            clean = {"input": item["input"], "output": item["output"]}
            f.write(json.dumps(clean, ensure_ascii=False) + "\n")
    print(f"저장 완료: {output_path} ({len(data)}개)")


OUTPUT_DIR = Path("../data/stepfather")

save_jsonl(set1, str(OUTPUT_DIR / "stepfather_dialogue_00.jsonl"))
save_jsonl(set2, str(OUTPUT_DIR / "stepfather_dialogue_01.jsonl"))

combined = set1 + set2
random.shuffle(combined)
save_jsonl(combined, str(OUTPUT_DIR / "stepfather_dialogue_combined.jsonl"))
print(f"\n합본 저장 완료: {len(combined)}개")

## 11. 최종 검증

In [ ]:
for fname in ["stepfather_dialogue_00.jsonl", "stepfather_dialogue_01.jsonl", "stepfather_dialogue_combined.jsonl"]:
    fpath = OUTPUT_DIR / fname
    with open(fpath, "r", encoding="utf-8") as f:
        lines = f.readlines()
    print(f"{fname}: {len(lines)}줄")
    for i, line in enumerate(lines):
        try:
            obj = json.loads(line)
            assert "input" in obj and "output" in obj
            assert len(obj["output"]) > 0
        except Exception as e:
            print(f"  ERROR at line {i}: {e}")
            break
    else:
        print(f"  -> 파싱 검증 통과")

print("\n=== 의미 축별 최종 샘플 ===")
axis_names_kr = {
    "movement_control":   "이동 통제",
    "question_dismissal": "질문 차단",
    "compliance_ack":     "복종 확인",
    "escape_boundary":    "탈출 경계",
    "past_probe":         "과거 탐문",
    "night_report":       "야간 보고",
}
for axis in AXIS_PROMPTS.keys():
    axis_items = [item for item in combined if item["_axis"] == axis]
    samples = random.sample(axis_items, min(3, len(axis_items)))
    print(f"\n--- {axis_names_kr.get(axis, axis)} ---")
    for s in samples:
        print(f"  플레이어: {s['input']}")
        print(f"  아더:     {s['output']}")
        print()